In [1]:
import pandas as pd

train_path = "file/HospitalDataSet1/data_cat10_annotated_train.txt"
eval_path = "file/HospitalDataSet1/data_cat10_annotated_eval.txt"
test_path = "file/HospitalDataSet1/data_cat10_annotated_test.txt"
train_output = "file/train.json"
eval_output = "file/eval.json"
test_output = "file/test.json"

In [2]:
import ml.config as config
import json

id2label = {}
with open('file/id2label.txt', 'r') as txt:
    for line in txt:
        ID, label = line.strip().split(' ')
        id2label[ID] = label

print(id2label)
with open('file/id2label.json', 'w') as f:
    json.dump(id2label, f, ensure_ascii=False, indent = 2)

label2id = {v:k for k, v in id2label.items()}
print(label2id)
with open('file/label2id.json', 'w') as f:
    json.dump(label2id, f, ensure_ascii=False, indent = 2)

{'1': '男科', '2': '皮肤科', '3': '其它', '4': '妇科', '5': '心血管内科', '6': '耳鼻喉科', '7': '消化内科', '8': '小儿内科', '9': '呼吸内科', '10': '内分泌科', '11': '神经内科'}
{'男科': '1', '皮肤科': '2', '其它': '3', '妇科': '4', '心血管内科': '5', '耳鼻喉科': '6', '消化内科': '7', '小儿内科': '8', '呼吸内科': '9', '内分泌科': '10', '神经内科': '11'}


In [5]:
def get_data(in_path, output_path):
    data = {}
    with open(in_path, "r") as f:
        for i, line in enumerate(f.readlines()):
            line = line.strip()
            line = line.split("\t")
            if i < 2:
                print(line)
            if len(line) != 2:
                continue
            label, text = line[0], line[1]
            if label not in data:
                data[label] = []
            data[label].append(text)
    for key in data.keys():
        print("label: %s, num: %d" % (id2label[key], len(data[key])))
    with open(output_path, mode="w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [6]:
get_data(train_path, train_output)
get_data(eval_path, eval_output)
get_data(test_path, test_output)

['5', '勃起 困难 一年 前 前列腺炎 住院 治疗 || 万艾可 能 给开 么']
['8', '四 个 半月 || 女 || 婴儿 || 体重 NUM 斤 || 之前 晚上 夜醒 一 到 两 次 吃奶 || 最近 晚上 频繁 夜醒 || 醒 了 大哭 || 抱 着 睡着 还 放 不下 || 孩子 舌苔 从 出生到现在 一直 很 厚 || 颜色 偏 黄白 || 大便 一天 一 到 两 次 无 奶瓣 、 泡沫 、 气味 偏臭 、 黄色 粘稠 状 || 全 母乳 喂养 || 一般 三 个 小时 左右 喂养 一次 || 白天 睡觉 也 爱哭 || 哭 了 不 醒']
label: 心血管内科, num: 1195
label: 小儿内科, num: 2050
label: 其它, num: 2916
label: 男科, num: 10613
label: 妇科, num: 1305
label: 耳鼻喉科, num: 912
label: 呼吸内科, num: 2443
label: 消化内科, num: 1501
label: 内分泌科, num: 1144
label: 神经内科, num: 1191
label: 皮肤科, num: 3103
['5', '大夫 我 爸 三 个 月 前 做 了 心 胀 支架 手术 || 手术 过程 中 支架 部位 出血 心 包 积液 （ 中量 ） 后 穿刺 抽 出去 血 || 住院 八 天 后 回家 调理 || 今天 三 个 多 月 来 附属 医院 做 心 彩超 结果 || 麻烦 大夫 给 看 一下']
['10', '本人 现 吃药 NUM 年 || 二甲双胍 缓释 片 早晚 各 NUM 片 || 血糖 NUM-NUM 左右 || 体重 NUM 公斤 || 身高 NUM . 今天 去 看 医生 || 医生 建议 加沙格列叮片 每天 早上 一片']
label: 心血管内科, num: 67
label: 内分泌科, num: 63
label: 呼吸内科, num: 137
label: 男科, num: 591
label: 皮肤科, num: 173
label: 神经内科, num: 66
label: 消化内科, num: 84
label: 妇科, num: 72
label: 其它, num: 162
labe

In [8]:
def data_to_csv(filepath):
    samples = []
    nanke = 0
    with open(filepath, 'r') as txt:
        for line in txt:
            ID, text = line.strip().split('\t')
            text = text.replace("||", "")
            label = id2label[ID]
            if label == "男科":
                if nanke >= 2000:
                    continue
                else:
                    nanke += 1
            sample = label+'\t'+text
            samples.append(sample)

    outfile = config.train_data_file
    if 'eval' in filepath:
        outfile = config.eval_data_file
    if 'test' in filepath:
        outfile = config.test_data_file

    with open(outfile, 'w') as csv:
        csv.write('label\ttext\n')
        for sample in samples:
            csv.write(sample)
            csv.write('\n')

    if 'train' in filepath:
        print("训练数据：%s" % str(len(samples)))
    elif 'test' in filepath:
        print("测试数据：%s" % str(len(samples)))
    else:
        print("验证数据：%s" % str(len(samples)))


In [9]:
data_to_csv(config.train_raw_file)
data_to_csv(config.eval_raw_file)
data_to_csv(config.test_raw_file)

训练数据：28373
验证数据：1580
测试数据：1580
